In [2]:
import folium
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [5]:
pattern_df = pd.read_parquet("../cta-stop-etl/out/pids/95.parquet")
# grab one trip to try stuff out
single_trip_df = pattern_df[pattern_df['unique_trip_vehicle_day'] == '7295.0235318404101004820292023-01-01']
single_trip_gdf = gpd.GeoDataFrame(
        single_trip_df,
        geometry=gpd.GeoSeries.from_xy(
            x=single_trip_df.loc[:, "lon"], y=single_trip_df.loc[:, "lat"], crs="EPSG:4326"
        ))

In [6]:
single_trip_points = [[point.xy[1][0], point.xy[0][0]] for point in single_trip_gdf.geometry ]

In [8]:
segments_df = pd.read_parquet("../cta-stop-etl/out/patterns/pid_95_segment.parquet")
segments_df['geometry'] = segments_df['geometry'].apply(wkt.loads)
segments_df = gpd.GeoDataFrame(segments_df, crs='epsg:4326')

In [10]:
pattern_segements_95 = gpd.GeoSeries(segments_df["geometry"]).simplify(tolerance=0.001)
pattern_segements_95 = pattern_segements_95.to_json()

In [11]:
m = folium.Map(
    location=[41.83491987636846, -87.62004994799588],
    zoom_start=12,
    zoomSnap=0.5,
    tiles=None,
    min_zoom=10,
    max_zoom=12,
    min_lat=41.6,
    max_lat=42.1,
    min_lon=-87.27481026390364,
    max_lon=-87.981026390364,
    max_bounds=True,
)
base_map = folium.FeatureGroup(name="Basemap", overlay=True, control=False)
folium.TileLayer(tiles="cartodb positron").add_to(base_map)
base_map.add_to(m)
for loc in single_trip_points:
    m.add_child(folium.Marker(location = loc))

folium.PolyLine(
    locations=single_trip_points,
    color='green',
    weight=2
).add_to(m)

folium.GeoJson(data=pattern_segements_95, style_function=lambda x: {"fillColor": "orange"}).add_to(m)

m
